### Data Preparation

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import os

# result.json contains flattened json (so list of dicts)
f = open(os.path.join(os.getcwd(), "..", "data_collection", "json", "flat_json_hiphop_rock_jazz.json"))
json_data = json.load(f)

# Convert list of dicts to dataframe
df = pd.DataFrame(json_data) 
df

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Drop unneeded columns
columns_to_drop = [
    "category.name",
    "category.playlist.id",
    "category.playlist.name",
    "category.playlist.track.id",
    "category.playlist.track.name",
    "category.playlist.track.album.id",
    "category.playlist.track.album.name",
    "category.playlist.track.artist"
    ]
df = df.drop(columns=columns_to_drop)

df = df.rename(columns={
    "category.id": "category",
    "category.playlist.track.feature.danceability": "feature_danceability",
    "category.playlist.track.feature.energy": "feature_energy",
    "category.playlist.track.feature.key": "feature_key",
    "category.playlist.track.feature.loudness": "feature_loudness",
    "category.playlist.track.feature.mode": "feature_mode",
    "category.playlist.track.feature.speechiness": "feature_speechiness",
    "category.playlist.track.feature.acousticness": "feature_acousticness",
    "category.playlist.track.feature.instrumentalness": "feature_instrumentalness",
    "category.playlist.track.feature.liveness": "feature_liveness",
    "category.playlist.track.feature.valence": "feature_valence",
    "category.playlist.track.feature.tempo": "feature_tempo",
    "category.playlist.track.feature.duration_ms": "feature_duration_ms",
    "category.playlist.track.feature.time_signature": "feature_time_signature"
})
df

,category,feature_danceability,feature_energy,feature_key,feature_loudness,feature_mode,feature_speechiness,feature_acousticness,feature_instrumentalness,feature_liveness,feature_valence,feature_tempo,feature_duration_ms,feature_time_signature
0,hiphop,0.849,0.424,5,-9.579,0,0.324,0.0635,0,0.0834,0.153,145.887,242966,4
1,hiphop,0.681,0.63,1,-5.585,1,0.0385,0.00383,0,0.139,0.183,151.951,161053,4
2,hiphop,0.711,0.611,1,-5.453,1,0.329,0.00575,0,0.231,0.144,134.14,252070,4
3,hiphop,0.723,0.516,11,-10.707,0,0.485,0.00311,1.26e-06,0.115,0.223,155.967,123077,4
4,hiphop,0.91,0.585,11,-7.572,0,0.257,0.0536,0,0.127,0.599,129.011,165067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18723,jazz,0.421,0.0952,6,-12.561,1,0.0479,0.931,0.000201,0.126,0.0773,109.698,177922,4
18724,jazz,0.503,0.491,0,-12.02,1,0.0295,0.0412,0.922,0.0965,0.489,166.105,263447,4
18725,jazz,0.644,0.594,5,-9.965,1,0.117,0.751,0.224,0.107,0.632,90.564,494467,4
18726,jazz,0.462,0.211,0,-13.396,1,0.0586,0.665,0.946,0.114,0.426,179.658,77190,3


In [4]:
from sklearn.model_selection import train_test_split

# Shuffle then split test and training data
df_target_shuffled = df.sample(frac=1, random_state=45)
train, test = train_test_split(df_target_shuffled, test_size=0.2, random_state=45, shuffle=False)
print("Number of rows in train: ", train.shape[0])
print("Number of rows in test: ", test.shape[0])

Number of rows in train:  14982
Number of rows in test:  3746


In [5]:
# sklearn takes the features and labels as seperate lists
# df needs to be split
def encode_target(df, target_column):

    df_mod = df.copy()
    map_to_int = {name: n for n, name in enumerate(df_mod["category"].unique())}
    df_mod["target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod)

train = encode_target(train, "category")

# Target to category mapping
train[["target", "category"]].head()

,target,category
5375,0,rock
1115,1,hiphop
10645,0,rock
15212,2,jazz
16506,2,jazz


In [6]:
train = train.astype(
    {
        "category": str,
        "feature_danceability": np.float32,
        "feature_energy": np.float32,
        "feature_key": np.int16,
        "feature_loudness": np.float32,
        "feature_mode": np.int16,
        "feature_speechiness": np.float32,
        "feature_acousticness": np.float32,
        "feature_instrumentalness": np.float32,
        "feature_liveness": np.float32,
        "feature_valence": np.float32,
        "feature_tempo": np.float32,
        "feature_duration_ms": np.int16,
        "feature_time_signature": np.int16,
        "target": np.int16,
    }
)
train.dtypes

category                     object
feature_danceability        float32
feature_energy              float32
feature_key                   int16
feature_loudness            float32
feature_mode                  int16
feature_speechiness         float32
feature_acousticness        float32
feature_instrumentalness    float32
feature_liveness            float32
feature_valence             float32
feature_tempo               float32
feature_duration_ms           int16
feature_time_signature        int16
target                        int16
dtype: object

In [7]:
train.pop("category")
train_target = train.pop("target")
dataset = tf.data.Dataset.from_tensor_slices((train.values, train_target.values))
dataset

<TensorSliceDataset shapes: ((13,), ()), types: (tf.float32, tf.int16)>

In [8]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 6.24000e-01  8.99000e-01  0.00000e+00 -5.70900e+00  1.00000e+00
  2.96000e-02  2.21000e-02  1.37000e-04  9.75000e-02  7.90000e-01
  1.04061e+02 -3.23500e+03  4.00000e+00], Target: 0
Features: [ 6.63000e-01  3.51000e-01  5.00000e+00 -1.20640e+01  0.00000e+00
  3.90000e-01  4.06000e-01  0.00000e+00  1.47000e-01  4.77000e-01
  1.18505e+02 -3.58000e+02  4.00000e+00], Target: 1
Features: [ 4.3200e-01  8.8600e-01  8.0000e+00 -3.9910e+00  1.0000e+00  3.4100e-02
  9.2000e-05  4.7400e-05  3.3800e-01  2.4800e-01  7.6040e+01 -2.0035e+04
  4.0000e+00], Target: 0
Features: [ 3.0900e-01  1.0400e-01  0.0000e+00 -2.3378e+01  1.0000e+00  4.2800e-02
  9.7600e-01  9.1800e-01  1.4900e-01  7.7900e-02  5.0329e+01 -2.1001e+04
  4.0000e+00], Target: 2
Features: [ 6.81000e-01  6.19000e-01  4.00000e+00 -8.62900e+00  1.00000e+00
  3.45000e-02  4.03000e-01  8.20000e-01  5.81000e-02  1.34000e-01
  1.38006e+02  3.92700e+03  4.00000e+00], Target: 2


In [9]:
import tensorflow.keras as ks
model = ks.models.Sequential([ks.layers.Dense(1)])
model.compile()
model.fit(dataset.batch(1))

ValueError: in user code:

    File "/home/luisp/development/spotify-genre-prediction/env/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/luisp/development/spotify-genre-prediction/env/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/luisp/development/spotify-genre-prediction/env/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/luisp/development/spotify-genre-prediction/env/lib/python3.8/site-packages/keras/engine/training.py", line 816, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/luisp/development/spotify-genre-prediction/env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
      

In [22]:
FEATURES = list(train.columns[1:14])
CATEGORICAL_FEATURES = ['feature_key', 'feature_mode', 'feature_time_signature']
NUMERIC_FEATURES = []
for column in FEATURES:
    if column not in CATEGORICAL_FEATURES:
        NUMERIC_FEATURES.append(column)

print("FEATURES: ", FEATURES)
print("NUMERIC_FEATURES: ", NUMERIC_FEATURES)
print("CATEGORICAL_FEATURES: ", CATEGORICAL_FEATURES)

FEATURES:  ['feature_danceability', 'feature_energy', 'feature_key', 'feature_loudness', 'feature_mode', 'feature_speechiness', 'feature_acousticness', 'feature_instrumentalness', 'feature_liveness', 'feature_valence', 'feature_tempo', 'feature_duration_ms', 'feature_time_signature']
NUMERIC_FEATURES:  ['feature_danceability', 'feature_energy', 'feature_loudness', 'feature_speechiness', 'feature_acousticness', 'feature_instrumentalness', 'feature_liveness', 'feature_valence', 'feature_tempo', 'feature_duration_ms']
CATEGORICAL_FEATURES:  ['feature_key', 'feature_mode', 'feature_time_signature']


In [25]:
train[NUMERIC_FEATURES].values

array([['0.624', '0.899', '-5.709', ..., '0.79', '104.061', '193373'],
       ['0.663', '0.351', '-12.064', ..., '0.477', '118.505', '196250'],
       ['0.432', '0.886', '-3.991', ..., '0.248', '76.04', '307645'],
       ...,
       ['0.798', '0.574', '-6.548', ..., '0.309', '155.964', '138587'],
       ['0.461', '0.858', '-6.598', ..., '0.564', '123.193', '259200'],
       ['0.56', '0.843', '-4.188', ..., '0.892', '152.598', '201240']],
      dtype=object)

In [26]:
tf.data.Dataset.from_tensor_slices(
    (
        tf.cast(train[CATEGORICAL_FEATURES].values, tf.int16),
        tf.cast(train[NUMERIC_FEATURES].values, tf.float32),
        tf.cast(train["target"].values, tf.int16)
    )
)

2021-12-12 15:19:35.532889: W tensorflow/core/framework/op_kernel.cc:1722] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to int16 is not supported


UnimplementedError: Cast string to int16 is not supported [Op:Cast]

In [9]:
feature_columns = []
for feature_name in CATEGORICAL_FEATURES:
  vocabulary = train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_FEATURES:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='feature_key', vocabulary_list=('0', '5', '8', '4', '9', '11', '2', '10', '1', '7', '6', '3'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='feature_mode', vocabulary_list=('1', '0'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='feature_time_signature', vocabulary_list=('4', '3', '1', '5'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='feature_danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='feature_energy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='feature_loudness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='feature_speechiness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='feature_acousticness', shape=(1,), default_value=None, dtype=tf

In [94]:
tf.data.Dataset.from_tensor_
# y contains list of target values
y_train = train["target"]
y_test = test["target"]
y_all = df_target_shuffled["target"]

# X contains dataframe with feature columns
X_train = train[features]
X_test = test[features]
X_all = df_target_shuffled[features]


In [90]:
# sklearn takes the features and labels as seperate lists
# df needs to be split
def encode_target(df, target_column):

    df_mod = df.copy()
    map_to_int = {name: n for n, name in enumerate(df_mod["category"].unique())}
    df_mod["target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod)

df_target = encode_target(df, "category")

# Target to category mapping
df_target[["target", "category"]].head()

,target,category
0,0,hiphop
1,0,hiphop
2,0,hiphop
3,0,hiphop
4,0,hiphop


In [91]:
# Possible values for category
targets = df_target["category"].unique()
targets

array(['hiphop', 'rock', 'jazz'], dtype=object)

### Train model

In [93]:
# List of feature names

features = list(train.columns[1:14])
features

['feature_danceability',
 'feature_energy',
 'feature_key',
 'feature_loudness',
 'feature_mode',
 'feature_speechiness',
 'feature_acousticness',
 'feature_instrumentalness',
 'feature_liveness',
 'feature_valence',
 'feature_tempo',
 'feature_duration_ms',
 'feature_time_signature']